In [ ]:
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
import liana as li
import anndata

sc.set_figure_params(dpi=300,dpi_save=300, format="svg",fontsize=10)

sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams["font.family"] = "Arial"
output_fol = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/241117_macrovascular_subcluster'
os.makedirs(output_fol,exist_ok=True)
sc.settings.figdir = output_fol


In [ ]:
uterus_adata = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')
ct_order=['Ciliated', 'Epithelial', 'Glandular', 'Lumenal',
         'Capillary','Lymphatic EC', 'Macrovascular',
          'Basophil', 'Dendritic', 'Myeloid','Proliferative myeloid','NK cell',  'T cell',
          'Matrix fibroblast','Vessel fibroblast','Trophoblast','Uterine smooth muscle', 'Vascular smooth muscle',
         ]
uterus_adata.obs['Cell Subtype'] = pd.Categorical(uterus_adata.obs['Cell Subtype'], categories=ct_order)
uterus_adata.obs['organ_tissue'] = 'Myometrium'


In [ ]:
macrovascular_adata = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/figures/subcluster/Macrovascular/Macrovascular_adata.gz.h5ad')
macrovascular_adata.uns['log1p']['base']=None
sc.tl.leiden(macrovascular_adata, resolution=0.1)
sc.pl.umap(macrovascular_adata,color='leiden',save='macrovascular_releiden.svg')
leiden_dict = {'0':'Arterial EC',
              '1':'Venous EC'}
macrovascular_adata.obs['Cell Subtype'] = [leiden_dict[x] for x in macrovascular_adata.obs['leiden']]
sc.pl.umap(macrovascular_adata,color='Cell Subtype',save='macrovascular_celltype.svg')
sc.tl.rank_genes_groups(macrovascular_adata,'Cell Subtype',method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(macrovascular_adata,dendrogram=False)

In [ ]:
jess_genes = {'Art':['COL8A1','GJA5','PDE3A','SETBP1','BMP6','MMP16','EXT1','HMCN1','CDH13','FAM189A1','ELN','TOX'],
              'Ven':['EBF1','NOSTRIN','TSHZ2','GALNT18'],
             }

sc.pl.dotplot(macrovascular_adata, jess_genes,groupby='Cell Subtype')

In [ ]:
endo_adata = uterus_adata[uterus_adata.obs['Lineage']=='Endothelial']
endo_adata.obs['Macro_sub'] = macrovascular_adata.obs['Cell Subtype']

In [ ]:
endo_adata.obs['Macro_sub']

In [ ]:
endo_adata.obs['Cell Subtype']= [x if x!='Macrovascular' else y for x,y in zip(endo_adata.obs['Cell Subtype'],endo_adata.obs['Macro_sub'])]
sc.pl.embedding(endo_adata,basis='X_umap_Endothelial',color='Cell Subtype',save='celltype.svg')
sc.pl.dotplot(endo_adata,jess_genes,groupby='Cell Subtype',save='genes.svg')
                            

In [ ]:
gene_ls = jess_genes['Art'] + jess_genes['Ven']
sc.pl.embedding(endo_adata,basis='X_umap_Endothelial',color=gene_ls,save='genes.svg',cmap='viridis'
          )

In [ ]:
sc.tl.rank_genes_groups(macrovascular_adata,'Cell Subtype',method='wilcoxon',pts=True)
sc.pl.rank_genes_groups_dotplot(macrovascular_adata,save='macrovascular_comparison.svg')
df = sc.get.rank_genes_groups_df(
                    macrovascular_adata, group="Arterial EC"
                )
df = df.rename({'pct_nz_group':'Arterial_%expressed',
                  'pct_nz_reference':'Venous_%expressed'},axis=1)
df.to_csv(f'{output_fol}/arterial_venous_degs.csv')